In [45]:
import os
import pandas as pd
import pickle as pkl
import sys
sys.path.append(os.path.abspath('../../../TMR_to_BF'))
from BF_checkers import *
from BF_properties import *

In [51]:
def bnet_to_int_func(filetoconv):
    node_num = {}
    in_edges = {}
    all_list = []
    all_list1 = []   
    c=0
    with open(filetoconv, 'r') as file1:
        for line in file1.readlines()[1:]:
            all_list += line.split(',')
            line = line.replace('(',' ( ').replace(')',' ) ').replace('!',' ! ')
            all_list1.append(line.split())
            node_num[line.split(',')[0]] = c
            c+=1
        ss = ''.join(all_list)
        ss = ss.replace('|', '').replace('!', '').replace('&', '').replace('(', '').replace(')', '').replace('[','').replace(']','')
        g = ss.split('\n')
        w = []
        for ele in g:
            w += ele.split()
        w = list(set(w))
        no_inp_nodes = []
        for item in w:
            if item not in node_num.keys():
                node_num[item] = c
                c+=1
                no_inp_nodes.append(item)
      
        for row in all_list1:
            a = [k for k in row if k in node_num.keys()]
            b = str(row).split(',')[0][2:]
            in_edges[node_num[b]] = list(set([node_num[k] for k in a]))
        for item in no_inp_nodes:
            in_edges[node_num[item]] = [node_num[item]]
    file1.close()

    file3 = open(filetoconv, 'r')
    g = file3.read()
    g = g.replace('&','and').replace('!','not ').replace('|', 'or')
    g1 = g.split('\n')
    g1.pop(0)
    if len(g1[-1]) == 0:
        g1.pop(-1)
    node_num_swap = {v: k for k, v in node_num.items()}
    BFint = []
    for i in range(len(node_num_swap)):
        if node_num_swap[i] not in no_inp_nodes:
            k = in_edges[i]
            z = [str(bin(i)[2:].zfill(len(k))) for i in range(2**len(k))]
            bfstr= ''
            ll = []
            for num in k:
                ll.append(node_num_swap[num].replace('/','_'))
            for item in z:
                for no in range(len(ll)):
                    exec("%s = %d" % (ll[no],int(item[no])))
                Q = eval(g1[i].split(',')[1][1:].replace('/','_'))
                bfstr = bfstr + str(int(Q))
            BFint.append(int(bfstr,2))
        else:
            BFint.append(1)
    file3.close()
    return node_num, in_edges, BFint, node_num_swap


def get_edgelist(inedges, int_func, node_num_swap, model_name, model_folder):
    frolist, tolist, signlist = [],[],[]
    edgelistdict = {'from':frolist, 'to':tolist, 'sign':signlist}
    for i in inedges:
        f = bin(int_func[i])[2:].zfill(2**len(inedges[i]))
        a = check_if(len(inedges[i]), f).is_UF()
        for j in range(len(a)):
            frolist.append(node_num_swap[inedges[i][j]] )
            tolist.append(node_num_swap[i])
            signlist.append(a[j])
    df = pd.DataFrame(edgelistdict)
    file_name_final = model_folder+ f'/{model_name}_edgelist.tsv'
    df.to_csv(file_name_final, sep="\t", index=False)
    return  df


def get_func_type(inedges, int_func, node_num_swap, model_name, model_folder):
    nodelist, inputlist, EFlist, UFlist, NCFlist = [],[],[],[],[]
    functypedict = {'Node': nodelist, 'No. of inputs':inputlist, 'EF':EFlist, 'UF':UFlist ,'NCF':NCFlist}
    for i in inedges:
        k = len(inedges[i])
        f = bin(int_func[i])[2:].zfill(2**k)
        nodelist.append(node_num_swap[i])
        inputlist.append(k)
        
        if check_if(k, f).is_EF():
            EFlist.append('Yes')
        else:
            EFlist.append('No')
            
        checkunate = check_if(k, f).is_UF()
        if checkunate == False:
             UFlist.append('No')
        else:
            UFlist.append('Yes')
        if check_if(k, f).is_NCF():
            NCFlist.append('Yes')
        else:
            NCFlist.append('No')
            
    df1 = pd.DataFrame(functypedict)
    df1.index = df1.index + 1
    file_name_final = model_folder+ f'/func_type_details_{model_name}.tsv'
    df1.to_csv(file_name_final, sep="\t", index=True)
    return df1

In [53]:
input_folder = "../input/published_models/"
output_base_folder = "../output/published_network_info/"

# Create the output base folder if it doesn't exist
os.makedirs(output_base_folder, exist_ok=True)

# Iterate over files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".bnet") and "model_" in file_name:
        print(file_name)
        x = file_name.split("_")[1]
        model_name = f"model_{x}"
        model_folder = os.path.join(output_base_folder, f"model_{x}")
        os.makedirs(model_folder, exist_ok=True)
        full_file_path = os.path.join(input_folder, file_name)
        output_bnet_to_int = bnet_to_int_func(full_file_path)
        int_func = output_bnet_to_int[2]
        inedges = output_bnet_to_int[1]
        node_num_swap = output_bnet_to_int[3]
        with open (model_folder + '/node_num_swap.pkl', 'wb') as file1:
            pkl.dump(node_num_swap, file1)
        with open (model_folder + '/inedges.pkl', 'wb') as file2:
            pkl.dump(inedges, file2)
        with open (model_folder + '/int_func.pkl', 'wb') as file3:
            pkl.dump(int_func, file3)
        get_edgelist(inedges, int_func, node_num_swap, model_name, model_folder)
        get_func_type(inedges, int_func, node_num_swap, model_name, model_folder)

model_85_final.bnet
model_200_final.bnet
model_99_final.bnet
model_88_final.bnet
model_40_final.bnet
model_22_final.bnet
model_10_final.bnet
model_208_final.bnet
model_8_final.bnet
model_63_final.bnet
model_31_final.bnet
model_109_final.bnet
model_69_final.bnet
model_133_final.bnet
model_7_final.bnet
model_110_final.bnet
model_61_final.bnet
model_67_final.bnet
model_23_final.bnet
model_95_final.bnet
model_100_final.bnet
model_86_final.bnet
model_212_final.bnet
model_74_final.bnet
